In [129]:
import pandas as pd
import numpy as np
import re

Energy = pd.read_excel('Energy Indicators.xls')

# Delete unnecesary columns and rows
Energy.drop(columns=['Unnamed: 0', 'Unnamed: 1'], inplace=True)
Energy.drop(Energy.index[0:17], inplace=True)
Energy.drop(Energy.index[227:], inplace=True)

# Formating the data. Use good column names and number units.
Energy.rename(columns={
    'Unnamed: 2': 'Country',
    'Unnamed: 3': 'Energy Supply',
    'Unnamed: 4': 'Energy Supply per Capita',
    'Unnamed: 5': '% Renewable'
}, inplace=True)
Energy.replace({'...': np.nan}, inplace=True)
Energy['Energy Supply'] = Energy['Energy Supply']*1000000
Energy.replace({
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
}, inplace=True)

# Fomatting countries names:
# Bolivia (Plurinational State of) -> Bolivia
# Spain16 -> Spain
def proccess_regex(pattern, row):
    result = re.findall(pattern, row)
    if result:
        return result[0]
    return row

Energy['Country'] = Energy['Country'].apply(
    lambda x: proccess_regex('([\w\s]+)(?:\ {1}\(.*\))', x)
)
Energy['Country'] = Energy['Country'].apply(
    lambda x: proccess_regex('([a-zA-Z]+)(?:\d+)', x)
)

# World bank datadrame
GDP = pd.read_csv('world_bank.csv')
GDP.drop(GDP.index[0:3], inplace=True)
GDP.rename(columns=GDP.iloc[0], inplace=True)
GDP.drop(GDP.index[0], inplace=True)
GDP.replace({
    "Korea, Rep.": "South Korea",
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
}, inplace=True)
cols = list(GDP.columns)
GDP.columns = cols[0:4] + [str(int(x)) for x in cols[4:]]
GDP.reset_index(inplace=True, drop=True)
GDP.rename(columns={'Country Name': 'Country'}, inplace=True)


ScimEn = pd.read_excel("scimagojr-3.xlsx")
# I can merge because both dataframe have a common column named Country
# if not an error is displayed
# No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False
j1 = pd.merge(ScimEn, Energy)
j2 = pd.merge(j1, GDP)

# Return the top 15 countries
j2 = j2[0:15]
columns = [
    'Country', 'Rank', 'Documents', 'Citable documents', 'Citations',
    'Self-citations', 'Citations per document', 'H index',
    'Energy Supply', 'Energy Supply per Capita', '% Renewable',
    '2006', '2007', '2008', '2009', '2010', '2011', '2012',
    '2013', '2014', '2015'
]
j2 = j2[columns]
j2.set_index('Country', inplace = True)
j2

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12
South Korea,10,11983,11923,114675,22595,9.57,104,1.100700e+10,221.0,2.279353,9.410199e+11,9.924316e+11,1.020510e+12,1.027730e+12,1.094499e+12,1.134796e+12,1.160809e+12,1.194429e+12,1.234340e+12,1.266580e+12
Italy,11,10964,10794,111850,26661,10.20,106,6.530000e+09,109.0,33.667230,2.202170e+12,2.234627e+12,2.211154e+12,2.089938e+12,2.125185e+12,2.137439e+12,2.077184e+12,2.040871e+12,2.033868e+12,2.049316e+12
